## 爬虫抓取与解析:链家网案例

来动手做一个练习，做到学以致用。
这次，我们来爬取链家网的一些内容，用的工具依旧是大家熟悉的requests和BeautifulSoup

### 1.准备工作

编写爬虫前的准备工作，我们需要导入用到的库，这里主要使用的是requests和BeautifulSoup两个。还有一个Time库，负责设置每次抓取的休息时间。

In [1]:
import requests
import time
from bs4 import BeautifulSoup

### 2.抓取列表页

开始抓取前当然应该了解一下目标网站URL结构咯。
链家网的二手房列表页面共有100个，URL结构为http://bj.lianjia.com/ershoufang/pg9/ 其中

bj表示城市
/ershoufang/是频道名称
pg9是页面码。
我们要抓取的是北京的二手房频道，所以前面的部分不会变，属于固定部分，后面的页面码需要在1-100间变化，属于可变部分。将URL分为两部分，前面的固定部分赋值给url，后面的可变部分使用for循环遍历页面。

In [2]:
# 设置列表页URL的固定部分
url = 'https://bj.lianjia.com/ershoufang/'
# 设置页面页的可变部分
page = ('pg')

这里提一个小小的醒，我们最好在http请求中设置一个头部信息，否则很容易被封ip。头部信息网上有很多现成的，也可以使用httpwatch等工具来查看。

In [4]:
#设置请求头部信息
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
'Accept':'text/html;q=0.9,*/*;q=0.8',
'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding':'gzip',
'Connection':'close',
'Referer':'http://www.baidu.com/link?url=_andhfsjjjKRgEWkj7i9cFmYYGsisrnm2A-TN3XZDQXxvGsM9k9ZZSnikW2Yds4s&amp;amp;wd=&amp;amp;eqid=c3435a7d00146bd600000003582bfd1f'
}

我们使用for循环生成1-100的数字，转化格式后与前面的URL固定部分拼成要抓取的URL。这里我们设置每两个页面间隔0.5秒。抓取到的页面保存在html中。

In [5]:
# 循环抓取列表页信息
for i in range(1,2):
    if i == 1:
        i = str(i)
        a = (url + page + i + '/')
        r = requests.get(url = a, headers = headers)
        html = r.content
    else:
        i = str(i)
        a = (url + page + i + '/')
        r = requests.get(url = a, headers = headers)
        html2 = r.content
        html = html + html2
    
    # 每次间隔1秒
    time.sleep(1)

### 页面解析

页面抓取的工作算是完成了，内容在html中，下一步就要进行页面解析了。我们依旧使用BeautifulSoup对页面进行解析。

In [6]:
# 解析抓取的页面内容
lj = BeautifulSoup(html, 'html.parser')

完成页面解析后就可以对页面中的关键信息进行提取了。下面我们分别对房源的总价，房源信息和关注度三部分进行提取。 把页面div标签中class=priceInfo的部分提取出来，并使用for循环将其中每个房源的总价数据存在tp中。

In [9]:
# 提取房源总价
price = lj.find_all('div',class_ = 'totalPrice')
# price
tp = []
for a in price:
    totalPrice = a.span.string
    tp.append(totalPrice)

来看看爬下来的房价数据

In [10]:
tp

['305',
 '830',
 '435',
 '935',
 '408',
 '265',
 '580',
 '680',
 '595',
 '950',
 '690',
 '274',
 '518',
 '455',
 '265',
 '430',
 '750',
 '520',
 '1490',
 '365',
 '718',
 '530',
 '520',
 '850',
 '392',
 '620',
 '588',
 '560',
 '550',
 '540']

提取房源信息和关注度的方法与提取房源价格的方法类似，下面是具体的代码，房源信息存储在hi中，关注度存储在fi中。

In [17]:
# 提取房源信息
# houseInfo = lj.find_all('div', class_ = 'houseInfo')
houseInfo = lj.find_all('div', attrs = {'class':'houseInfo'})
# houseInfo
hi = []
for b in houseInfo:
    house = b.get_text()
    hi.append(house)

# hi

来看看房源信息

In [19]:
for item in hi:
    print(item)

福苑小区 /3室1厅/82.02平米/南 北/精装/无电梯
今日家园 /3室2厅/141.31平米/西 北/精装/有电梯
安宁里南区 /2室1厅/69.51平米/东 西/简装/无电梯
澳洲康都 /3室2厅/129.92平米/西南/简装/有电梯
天竺花园 /3室2厅/108.97平米/南 北/简装/无电梯
惠泽家园 /2室1厅/90.22平米/南 北/简装/无电梯
金鱼池东区 /2室1厅/70.42平米/南 北/简装/无电梯
丽泽雅园 /2室1厅/107.59平米/西南/精装/有电梯
青秀城 /2室1厅/85.91平米/南 北/简装/有电梯
依莲轩 /2室1厅/108.9平米/西南/精装/有电梯
青秀城 /3室1厅/90.6平米/南 北/精装/有电梯
加州水郡三期 /2室1厅/87.77平米/西/精装/有电梯
优筑 /2室1厅/77.51平米/东北/精装/有电梯
青秀雅苑 /2室1厅/89.89平米/南 北/简装/有电梯
德露苑 /2室1厅/71.4平米/南/简装/无电梯
育芳园 /3室1厅/76.1平米/南 北/简装/无电梯
庄维花园 /3室1厅/135.88平米/东南/简装/有电梯
明月嘉园 /2室1厅/83.19平米/东北/简装/有电梯
丽都水岸 /3室1厅/139.7平米/南 西/精装/有电梯
龙华园 /2室1厅/65.94平米/南 北/毛坯/无电梯
庄维多摩市二期 /2室2厅/116.49平米/西南/精装/有电梯
郁花园二里 /2室2厅/107.12平米/南 北/精装/无电梯
怡海花园富泽园 /2室1厅/87.71平米/南 北/简装/有电梯
奥林匹克花园三期 /3室2厅/136.98平米/南 北/精装/有电梯
粮食局宿舍 /2室1厅/66.6平米/南 北/简装
文体路38号院 /3室1厅/103.6平米/南 西 东北/简装/有电梯
国美第一城3号院 /2室1厅/104.38平米/东南/其他/有电梯
龙腾苑六区 /3室2厅/116.9平米/东 南 北/精装/无电梯
三环新城8号院 /2室1厅/96.72平米/西南/精装/有电梯
新华街三里 /2室2厅/99.85平米/南 北/精装/无电梯


In [46]:
# 提取房源关注度
followInfo = lj.find_all('div', attrs = {'class':'followInfo'})

followInfo

# fi = []
# for c in followInfo:
#     follow = c.get_text()
#     fi.append(follow)

# fi

<div class="followInfo">44人关注<span class="divide">/</span>14次带看<div class="tag"><span class="vr">VR房源</span><span class="taxfree">房本满五年</span><span class="haskey">随时看房</span></div><div class="priceInfo"><div class="totalPrice"><span>305</span>万</div><div class="unitPrice" data-hid="101103288789" data-price="37187" data-rid="1111027374435"><span>单价37187元/平米</span></div></div></div>

再来看看关注度状况

In [36]:
for item in fi:
    print(item)

44人关注/14次带看VR房源房本满五年随时看房305万单价37187元/平米
82人关注/13次带看VR房源房本满五年随时看房830万单价58737元/平米
123人关注/14次带看VR房源房本满五年随时看房435万单价62581元/平米
62人关注/17次带看近地铁VR房源房本满五年随时看房935万单价71968元/平米
82人关注/14次带看VR房源房本满五年随时看房408万单价37442元/平米
61人关注/14次带看VR房源房本满五年随时看房265万单价29373元/平米
133人关注/23次带看近地铁VR房源房本满五年随时看房580万单价82363元/平米
50人关注/15次带看VR房源房本满五年随时看房680万单价63203元/平米
37人关注/17次带看VR房源房本满五年随时看房595万单价69259元/平米
23人关注/13次带看近地铁VR房源房本满五年随时看房950万单价87236元/平米
42人关注/17次带看VR房源房本满五年随时看房690万单价76159元/平米
74人关注/23次带看VR房源房本满五年随时看房274万单价31218元/平米
36人关注/17次带看近地铁VR房源房本满五年随时看房518万单价66831元/平米
61人关注/16次带看VR房源房本满五年随时看房455万单价50618元/平米
44人关注/13次带看VR房源房本满五年随时看房265万单价37115元/平米
22人关注/14次带看近地铁VR房源房本满五年随时看房430万单价56505元/平米
45人关注/19次带看近地铁VR房源房本满五年随时看房750万单价55196元/平米
99人关注/3次带看近地铁VR房源房本满五年520万单价62508元/平米
108人关注/2次带看VR房源房本满五年1490万单价106658元/平米
69人关注/13次带看近地铁VR房源房本满五年365万单价55354元/平米
15人关注/15次带看近地铁VR房源房本满两年718万单价61637元/平米
11人关注/13次带看近地铁VR房源房本满五年530万单价49478元/平米
37人关注/14次带看近地铁VR房源房本满五年520万单价59287元/平米
28人关注/15次带看VR房源房本满五年850万单价62053元/平米
6人关注/13次带看VR房源房本满五年392万单价58859元/

### 清洗数据并整理到数据表中

我们将之前爬取到的信息进行汇总，并导入pandas之中生成数据表。便于后面的分析。

In [38]:
# 导入pandas库
import pandas as pd
# 创建数据表
house = pd.DataFrame({'totalprice':tp, 'houseinfo':hi, 'followinfo':fi})
# 查看数据表的内容
house.head()

,totalprice,houseinfo,followinfo
0,305,福苑小区 /3室1厅/82.02平米/南 北/精装/无电梯,44人关注/14次带看VR房源房本满五年随时看房305万单价37187元/平米
1,830,今日家园 /3室2厅/141.31平米/西 北/精装/有电梯,82人关注/13次带看VR房源房本满五年随时看房830万单价58737元/平米
2,435,安宁里南区 /2室1厅/69.51平米/东 西/简装/无电梯,123人关注/14次带看VR房源房本满五年随时看房435万单价62581元/平米
3,935,澳洲康都 /3室2厅/129.92平米/西南/简装/有电梯,62人关注/17次带看近地铁VR房源房本满五年随时看房935万单价71968元/平米
4,408,天竺花园 /3室2厅/108.97平米/南 北/简装/无电梯,82人关注/14次带看VR房源房本满五年随时看房408万单价37442元/平米


很尴尬的是，大家看得到，很多信息是糊在一块的，不能直接使用，所以咱们再做一些数据提取和清洗的工作。如房源信息，在表中每个房源的小区名称，户型，面积，朝向等信息都在一个字段中，无法直接使用。需要先进行分列操作。这里的规则比较明显，每个信息间都是以竖线分割的，因此我们只需要以竖线进行分列即可。

In [39]:
# 对房源信息进行分列
houseinfo_split = pd.DataFrame((x.split('/') for x in house.houseinfo), index = house.index, columns= ['xiaoqu','huxing','mianji','chaoxiang','zhuangxiu','dianti'])

现在再来看看我们整理好的数据

In [41]:
# 查看分列结果
houseinfo_split.head()

,xiaoqu,huxing,mianji,chaoxiang,zhuangxiu,dianti
0,福苑小区,3室1厅,82.02平米,南 北,精装,无电梯
1,今日家园,3室2厅,141.31平米,西 北,精装,有电梯
2,安宁里南区,2室1厅,69.51平米,东 西,简装,无电梯
3,澳洲康都,3室2厅,129.92平米,西南,简装,有电梯
4,天竺花园,3室2厅,108.97平米,南 北,简装,无电梯


把拆分后的数据拼接回原始数据中

In [42]:
# 将分列结果拼接回原始数据表
house = pd.merge(house,houseinfo_split,right_index = True, left_index = True )

In [44]:
house.head()

,totalprice,houseinfo,followinfo,xiaoqu,huxing,mianji,chaoxiang,zhuangxiu,dianti
0,305,福苑小区 /3室1厅/82.02平米/南 北/精装/无电梯,44人关注/14次带看VR房源房本满五年随时看房305万单价37187元/平米,福苑小区,3室1厅,82.02平米,南 北,精装,无电梯
1,830,今日家园 /3室2厅/141.31平米/西 北/精装/有电梯,82人关注/13次带看VR房源房本满五年随时看房830万单价58737元/平米,今日家园,3室2厅,141.31平米,西 北,精装,有电梯
2,435,安宁里南区 /2室1厅/69.51平米/东 西/简装/无电梯,123人关注/14次带看VR房源房本满五年随时看房435万单价62581元/平米,安宁里南区,2室1厅,69.51平米,东 西,简装,无电梯
3,935,澳洲康都 /3室2厅/129.92平米/西南/简装/有电梯,62人关注/17次带看近地铁VR房源房本满五年随时看房935万单价71968元/平米,澳洲康都,3室2厅,129.92平米,西南,简装,有电梯
4,408,天竺花园 /3室2厅/108.97平米/南 北/简装/无电梯,82人关注/14次带看VR房源房本满五年随时看房408万单价37442元/平米,天竺花园,3室2厅,108.97平米,南 北,简装,无电梯


使用相同的方法对房源关注度字段进行分列和拼接操作。这里的分列规则是斜杠。

In [ ]:
# 对房源关注度进行分列
followinfo_split = pd.DataFrame((x.split('/') for x in house.followinfo),index=house.index,columns=['guanzhu','daikan','fabu'])